In [ ]:
from keras import backend as K
from keras.engine.topology import Layer
from keras.layers.core import Dropout, Reshape
from keras.layers.convolutional import ZeroPadding2D
from keras.models import Sequential
import numpy as np


def test_layer(layer, x):
    # Adjust layer input_shape to x.shape
    layer_config = layer.get_config()
    layer_config["input_shape"] = x.shape
    layer = layer.__class__.from_config(layer_config)
    model = Sequential()
    model.add(layer)
    # compileを行うことでモデル構築をチェック
    model.compile("rmsprop", "mse")
    _x = np.expand_dims(x, axis=0)  # Add dimension for batch size

    # predictを行うことで計算グラフ内にデータを通したときにうまく動くかをチェックしている
    return model.predict(_x)[0]


class LocalResponseNormalization(Layer):
    
    def __init__(self, n=5, alpha=0.0005, beta=0.75, k=2, **kwargs):
        # ハイパーパラメータの設定
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.k = k
        super(LocalResponseNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        # 層が学習可能な重みを持つ場合はここで初期化処理を行う
        # 今回は重みを持たないためここでは何もしない
        super(LocalResponseNormalization, self).build(input_shape)

    def call(self, x):
        # 伝搬処理で行われる処理を実装する
        # バッチサイズに依存しない処理にしておく必要がある
        squared = K.square(x)
        # WITHIN_CHANNEL Normalization
        average = K.pool2d(squared, (self.n, self.n), strides=(1, 1),
                           padding="same", pool_mode="avg")
        denom = K.pow(self.k + self.alpha * average, self.beta) # 要素ごとの指数乗
        return x / denom

    def compute_output_shape(self, input_shape):
        # この層を通過したあとの形状を出力する
        # 今回の層では行列サイズの変更はないためそのまま
        return input_shape


# Kerasで提供されている層に対してテストをかける
x = np.random.randn(10, 10)
layer = Dropout(0.5)
y = test_layer(layer, x)
assert(x.shape == y.shape)

x = np.random.randn(10, 10, 3)
layer = ZeroPadding2D(padding=(1, 1))
y = test_layer(layer, x)
assert(x.shape[0] + 2 == y.shape[0])
assert(x.shape[1] + 2 == y.shape[1])

x = np.random.randn(10, 10)
layer = Reshape((5, 20))
y = test_layer(layer, x)
assert(y.shape == (5, 20))

# test custom layer
x = np.random.randn(225, 225, 3)
layer = LocalResponseNormalization()
y = test_layer(layer, x)
assert(x.shape == y.shape)